## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
### First, I'll compute the camera calibration and distortion coefficients using chessboard images

The code in the cell below is to extract object points and image points for camera calibration, and save the drawChessboardCorners picture to camera_cal folder. 
The starter code from https://github.com/udacity/CarND-Camera-Calibration/blob/master/camera_calibration.ipynb

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib qt
import scipy.misc
import os
import re
print(cv2.IMREAD_COLOR)
print(cv2.IMREAD_GRAYSCALE)
print(cv2.IMREAD_UNCHANGED)

1
0
-1


In [2]:
def get_camera_parameters(camera_cal_folder="../camera_cal/calibration*.jpg", img_size=(1280, 720)):
    
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9, 3), np.float32)
    objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)
    
    # Arrays to store object points and image points from all the images
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane
    
    # Make a list of calibration images
    images = glob.glob(camera_cal_folder)
    
    miss_corners = 0 
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            # Draw and display the corners
            #img = cv2.drawChessboardCorners(img, (9,6), corners, ret)  
            #write_name = '../camera_cal/'+'corners_found_'+str(idx)+'.jpg'
            #cv2.imwrite(write_name, img)
            #cv2.imshow('img',img)
            #cv2.waitKey(500)
        else:
            print(fname + ' cannot get the corners.')
            miss_corners += 1
    print("{} number images cannot get the corners.".format(miss_corners))
    
    # Finds the camera intrinsic and extrinsic parameters from several views of a calibration pattern. 
    # retval, cameraMatrix, distCoeffs, rvecs, tvecs
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    return ret, mtx, dist, rvecs, tvecs

#ret, mtx, dist, rvecs, tvecs = get_camera_parameters("../camera_cal/calibration*.jpg")
#print(len(objpoints))
#print(len(imgpoints))
#print('Done')

def get_undistort(img="", mtx="", dist="", dst=None, newCameraMatrix=""):
    #https://docs.opencv.org/2.4/modules/imgproc/doc/geometric_transformations.html
    return cv2.undistort(img, mtx, dist, dst, newCameraMatrix)

Use the above objpoints and imgpoints to calculate camera calibration and distortion coefficients.

In [3]:
import pickle
%matplotlib inline

# Test undistortion on an image
img = cv2.imread('../camera_cal/calibration2.jpg')
img_size = (img.shape[1], img.shape[0])

# Get camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = get_camera_parameters()

# Get the undistortion picture
dst = get_undistort(img, mtx, dist, None, mtx)
#print("dst = ", dst)

cv2.imwrite('../output_images/calibration2_undistort.jpg', dst)

print('Test done')

../camera_cal/calibration5.jpg cannot get the corners.
../camera_cal/calibration4.jpg cannot get the corners.
../camera_cal/calibration1.jpg cannot get the corners.
3 number images cannot get the corners.
Test done


In [4]:
test_filename_list = os.listdir("../test_images/")

# Get camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = get_camera_parameters()

for fname in test_filename_list:
    img = cv2.imread("../test_images/"+fname)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    #print(dst.shape)
    cv2.imwrite('../output_images/'+'undistort_'+fname, dst)

print('Test done')

../camera_cal/calibration5.jpg cannot get the corners.
../camera_cal/calibration4.jpg cannot get the corners.
../camera_cal/calibration1.jpg cannot get the corners.
3 number images cannot get the corners.
Test done


### Secondly,  use color transforms, gradients, etc., to create a thresholded binary image.

In [5]:
# Define a function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Convert to grayscale
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #print("gray = ", gray)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
        #print("x abs_sobel = ", abs_sobel)
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    # Rescale back to 8 bit integer
    #scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # if use the np.uint8, the scipy.misc.imsave cannot save it
    scaled_sobel = 255*abs_sobel/np.max(abs_sobel)
    #scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    #print("abs_sobel_thresh = ", binary_output)
    return binary_output

# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255
    #gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # if use the np.uint8, the scipy.misc.imsave cannot save it
    gradmag = gradmag/scale_factor 
    #gradmag = np.uint8(gradmag/scale_factor)
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    #print("mag_thresh = ", binary_output)
    return binary_output

# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    #print("dir_threshold = ", binary_output)
    return binary_output

In [6]:
image = cv2.imread('../challenge_10sec/challenge_10sec-0006.jpg')
print(image.shape[1], image.shape[0])
print("image = ", type(image))
# Choose a larger odd number to smooth gradient measurements
ksize = 3

# Apply each of the thresholding functions
gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(20, 100))
# https://stackoverflow.com/questions/902761/saving-a-numpy-array-as-an-image
scipy.misc.imsave('../output_images/gradx_challenge_10sec-0006_xygrad.jpg', gradx)
#cv2.imwrite('../output_images/gradx_signs_vehicles_xygrad.jpg', gradx)

grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(20, 100))
scipy.misc.imsave('../output_images/grady_challenge_10sec-0006_xygrad.jpg', grady)

mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(30, 100))
scipy.misc.imsave('../output_images/mag_binary_challenge_10sec-0006_xygrad.jpg', mag_binary)

dir_binary = dir_threshold(image, sobel_kernel=ksize, thresh=(0.7, 1.3))
print("dir_binary = ", type(dir_binary))

scipy.misc.imsave('../output_images/dir_binary_challenge_10sec-0006_xygrad.jpg', dir_binary)

combined = np.zeros_like(dir_binary)
#combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 255

#print("combined = ", combined.astype(np.uint8))
print("combined = ", combined)
#scipy.misc.imsave('../output_images/all_comb_binary_signs_vehicles_xygrad.jpg', combined)
cv2.imwrite('../output_images/all_comb_binary_challenge_10sec-0006.jpg', combined)
#x = cv2.imread('../output_images/all_comb_binary_signs_vehicles_xygrad.jpg')
#print("x = ", x)
print('Test done')

1280 720
image =  <class 'numpy.ndarray'>
dir_binary =  <class 'numpy.ndarray'>
combined =  [[   0.    0.    0. ...,    0.    0.    0.]
 [   0.    0.    0. ...,    0.    0.    0.]
 [   0.    0.    0. ...,    0.    0.    0.]
 ..., 
 [   0.    0.    0. ...,  255.    0.    0.]
 [   0.    0.    0. ...,  255.    0.    0.]
 [   0.    0.    0. ...,    0.    0.    0.]]
Test done


use test_images folder to test calibration images for binary image

In [7]:
"""
def combined_binary(image, ksize=3, color_thresh=True):
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(20, 100))
    
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    s_thresh = (150, 255)
    s_condition = (S >= s_thresh[0]) & (S <= s_thresh[1])
    
    color_combined = np.zeros_like(gradx)
    color_combined[(gradx == 1) | (s_condition == 1)] = 255
    #color_combined[(s_condition == 1)] = 255

    return color_combined
    
"""
def combined_binary(image, ksize=3, color_thresh=True):
    # gradient thresh
    #gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(12, 100))
    #grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(24, 100))
    #gradient_combined = np.zeros_like(grady)
    #gradient_combined[(gradx == 1) & (grady == 1)] = 1
    
    # Good
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(10, 100))
    dir_binary = dir_threshold(image, sobel_kernel=ksize, thresh=(0.2, 1.2))
    gradient_combined = np.zeros_like(dir_binary)
    gradient_combined[(gradx == 1) & (dir_binary == 1)] = 1

    #mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(20, 100))
    #dir_binary = dir_threshold(image, sobel_kernel=ksize, thresh=(0.7, 1.3))
    #gradient_combined = np.zeros_like(dir_binary)
    #gradient_combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    # color thresh
    if color_thresh == True:
        img = np.copy(image)
        hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
        s_channel = hls[:,:,2]
        s_binary = np.zeros_like(s_channel)
        s_binary[(s_channel >= 80) & (s_channel <= 170)] = 1
        color_gradient_combined = np.zeros_like(gradient_combined)
        color_gradient_combined[(s_binary == 1) | (gradient_combined == 1)] = 1
    
    if color_thresh == True:
        color_gradient_combined = color_gradient_combined * 255
        #print("color_gradient_combined = ", color_gradient_combined)
        return color_gradient_combined 
    else:
        gradient_combined = gradient_combined * 255
        return gradient_combined



In [8]:
test_filename_list = os.listdir("../challenge_10sec/")
folder_path = "../challenge_10sec/"
color_thresh=True

for fname in test_filename_list:
    
    #only handle the "color_gradient_combined.*" files
    #m = re.match("undistort_", fname)
    #if m is None:
    #    continue
            
    print("fn = ", fname)
    image = cv2.imread(folder_path+fname)
    result = combined_binary(image, ksize=3, color_thresh=True)
    #print("result = ", result)
    if color_thresh == True:
        #Do not use cv2.imwrite(), or it will be none.
        #scipy.misc.imsave('../output_images/color_gradient_combined_'+ fname, result)
        cv2.imwrite('../output_images/10sec_combined_binary'+ fname, result)
    else:
        #scipy.misc.imsave('../output_images/gradient_combined_'+ fname, result)
        cv2.imwrite('../output_images/10sec_gradient_combined_'+ fname, result)

print("Test Done")

fn =  challenge_10sec-0008.jpg
fn =  challenge_10sec-0006.jpg
fn =  challenge_10sec-0005.jpg
fn =  challenge_10sec-0004.jpg
fn =  challenge_10sec-0007.jpg
fn =  challenge_10sec-0003.jpg
fn =  challenge_10sec-0001.jpg
fn =  challenge_10sec-0002.jpg
Test Done


As the source picure is not good, for the test1.jpg, test4.jpg, test6.jpg, the combined_binary of test is not enough good, other test picuture is good.

### Thirdly, Apply a perspective transform to rectify binary image

Get the coordinate of src and dest

In [9]:
#refer to the Perspective Transform section in https://cmlpr.github.io/blog/2017/03/14/advanced-lane-lines,  
def get_src_des_coordinate():
    box_width = 0.405 
    mid_width = 0.092 
    height_pct = 0.663
    bottom_trim = 0.935
    
    src = np.float32([[img.shape[1] * (.5 - mid_width / 2), img.shape[0] * height_pct],
                    [img.shape[1] * (.5 + mid_width / 2), img.shape[0] * height_pct],
                    [img.shape[1] * (.5 + box_width / 2), img.shape[0] * bottom_trim],
                    [img.shape[1] * (.5 - box_width / 2), img.shape[0] * bottom_trim]])
    
    offset = img.shape[1] * 0.3
    
    dst = np.float32([[offset, 0],
                    [img.shape[1] - offset, 0],
                    [img.shape[1] - offset, img.shape[0]],
                    [offset, img.shape[0]]]) 
    return src, dst

src, dst = get_src_des_coordinate()
print(src)
print(dst)

[[ 581.11999512  477.35998535]
 [ 698.88000488  477.35998535]
 [ 899.20001221  673.20001221]
 [ 380.79998779  673.20001221]]
[[ 384.    0.]
 [ 896.    0.]
 [ 896.  720.]
 [ 384.  720.]]


Get the perspective Matrix, then get the birds eye view picture

In [10]:
def perspective_M():
    #src = np.float32([[588, 446], [691, 446], [1126, 673], [154, 673]])
    #refer to https://chatbotslife.com/self-driving-cars-advanced-computer-vision-with-opencv-finding-lane-lines-488a411b2c3d
    src = np.float32([[581, 477], [699, 477], [896, 675], [384, 675]])
    dst = np.float32([[384, 0], [896, 0], [896, 720], [384, 720]]) 
    M = cv2.getPerspectiveTransform(src, dst)
    return M

def get_birds_eye_view(image="", M="", dsize=(1280, 720), flags=cv2.INTER_LINEAR):
    warped = cv2.warpPerspective(image, M, dsize, flags=cv2.INTER_LINEAR)
    return warped

def test_get_birds_eye_view(folder_path=""):
    test_filename_list = os.listdir(folder_path)
    M = perspective_M()
    
    for fname in test_filename_list:
        #only handle the "color_gradient_combined.*" files
        m = re.match("10sec_combined_binary", fname)
        if m is None:
            continue
        
        image = cv2.imread(folder_path+fname)
        #print("image.shape = ", image.shape)
        print("image.ndim = ", image.ndim)
        print("image = ", image)
        h, w = image.shape[0], image.shape[1]     
        warped = get_birds_eye_view(image, M, (w,h), flags=cv2.INTER_LINEAR)
        print("warped.ndim = ", warped.ndim)
        print("warped = ", warped)
        #print("warped.shape = ", warped.shape)
        #scipy.misc.imsave('../output_images/birds_eye_view_'+ fname, warped)
        cv2.imwrite('../output_images/10sec_birds_eye_view_'+ fname, warped)
        
    print("Done")
    
#test_get_birds_eye_view(folder_path="../output_images/")

### Fourth, detect lane pixels and fit to the lane boundary

In [11]:
# The most of code from Udacity lesson(Advanced Computer Vision). 
last_leftx_base = []
last_rightx_base = []
first_compute_leftx_base = True

last_left_fit = []
last_right_fit = []

def find_lines(binary_warped, nwindows=4, margin=40, minpix=50):
    
    global last_leftx_base
    global last_rightx_base
    global first_compute_leftx_base 
    
    global last_left_fit
    global last_right_fit
    
    leftx_base = 0
    rightx_base = 0
    
    #histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    histogram = np.sum(binary_warped[:,:], axis=0)
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    #print("leftx_base = ", leftx_base)
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    '''
    if first_compute_leftx_base == True:
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        #midpoint = np.int(histogram.shape[0]/2)
        #leftx_base = np.argmax(histogram[:midpoint])
        #rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        # I think, it should use the 700, is not the midpoint(640)
        rightx_off = midpoint - leftx_base
        rightx_base = midpoint + rightx_off
        
        last_leftx_base = leftx_base
        last_rightx_base = rightx_base
        first_compute_leftx_base = False
        print("1st")
        print("leftx_base = ", leftx_base)
        print("rightx_base = ", rightx_base)
        print("- " * 10)
    else:
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        # Number 50 is only threshold value
        if abs(leftx_base - last_leftx_base) > 40:
            leftx_base = last_leftx_base
        else:
            last_leftx_base = leftx_base
        rightx_off = midpoint - leftx_base
        rightx_base = midpoint + rightx_off
        print("leftx_base = ", leftx_base)
        print("rightx_base = ", rightx_base)
        print("- " * 10)
    '''
    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Create an output image to draw on and visualize the result
    # out_img = np.dstack((binary_warped, binary_warped, binary_warped)) * 255    
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window + 1) * window_height
        win_y_high = binary_warped.shape[0] - window * window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0, 255, 0), 2) 
        cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0, 255, 0), 2)
        
        # Identify the nonzero pixels in x and y within the window 
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
        
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    #print("lefty = ", lefty)
    #print("len(righty) = ", len(righty))
    # Fit a second order polynomial to each
    #if (len(lefty) == 0) | (len(righty) == 0):
    #    print("come in")
    #    left_fit,  right_fit = last_left_fit, last_right_fit
    #    print("come in left_fit = ", left_fit)
    #    print("come in right_fit= ", right_fit)
    #    print("come in last_left_fit = ", last_left_fit)
    #    print("come in last_right_fit = ", last_right_fit)
    #else:
    #    left_fit = np.polyfit(lefty, leftx, 2)
    #    right_fit = np.polyfit(righty, rightx, 2)
    #    last_left_fit, last_right_fit = left_fit,  right_fit
    #print("lefty = ", lefty)
    #print("righty = ", righty)
    #print("= " * 10)
    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    return left_fit, right_fit, left_lane_inds, right_lane_inds, nonzerox, nonzeroy


def search_around_poly(left_fit, right_fit, left_lane_inds, right_lane_inds, nonzerox, nonzeroy, 
                       binary_warped, margin=100):
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    
    return result

def get_lane_poly(folder_path=""):
    test_filename_list = os.listdir(folder_path)
    
    for fname in test_filename_list:
        #only handle the "birds_eye_view_.*" files
        m = re.match("10sec_birds_eye_view_", fname)
        if m is None:
            continue
        
        #if there is not 0, it hits the "error: /feedstock_root/build_artefacts/opencv_1495915003231/work/opencv-3.2.0/modules/core/src/matrix.cpp:1104: error: (-215) cn <= 4 in function scalarToRawData"
        # cv2.IMREAD_GRAYSCALE is 0
        #image = cv2.imread(folder_path+fname, cv2.IMREAD_GRAYSCALE)
        print("fn = ", fname)
        image = cv2.imread(folder_path+fname)
        print("image(BGR) = ", image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        print("image(GRAY) = ", image)

        #break
        
        #Do not work
        #image = mpimg.imread(folder_path+fname, 0)

        left_fit, right_fit, left_lane_inds, right_lane_inds, nonzerox, nonzeroy = find_lines(image)
        result = search_around_poly(left_fit, right_fit, left_lane_inds, right_lane_inds, nonzerox, nonzeroy, image)
        #scipy.misc.imsave('../output_images/search_around_poly_'+ fname, result)
        cv2.imwrite('../output_images/10sec_search_around_poly_'+ fname, result)
        
    print("Done")
    
#get_lane_poly(folder_path="../output_images/")

### Fifth, determine the curvature of the lane and vehicle position with respect ot center

In [12]:
def radius_curvature(binary_warped, left_fit, right_fit):

    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30./720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    y_eval = np.max(ploty)
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curvature =  ((1 + (2*left_fit_cr[0] *y_eval*ym_per_pix + left_fit_cr[1])**2) **1.5) / np.absolute(2*left_fit_cr[0])
    right_curvature = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    # Calculate vehicle center
    #left_lane and right lane bottom in pixels
    left_lane_bottom = (left_fit[0]*y_eval)**2 + left_fit[0]*y_eval + left_fit[2]
    right_lane_bottom = (right_fit[0]*y_eval)**2 + right_fit[0]*y_eval + right_fit[2]
    
    # Lane center as mid of left and right lane bottom                        
    lane_center = (left_lane_bottom + right_lane_bottom)/2.
    center_image = binary_warped.shape[1] / 2.
    center = (lane_center - center_image)*xm_per_pix #Convert to meters
    position = "left" if center < 0 else "right"
    center = "Vehicle is {:.2f}m {}".format(center, position)
    
    # Now our radius of curvature is in meters
    return left_curvature, right_curvature, center

def get_radius_curvature(folder_path=""):
    test_filename_list = os.listdir(folder_path)
    
    for fname in test_filename_list:
        #only handle the "birds_eye_view_.*" files
        m = re.match("birds_eye_view_", fname)
        if m is None:
            continue
        
        # if there is not 0, it hits the "error: /feedstock_root/build_artefacts/opencv_1495915003231/work/opencv-3.2.0/modules/core/src/matrix.cpp:1104: error: (-215) cn <= 4 in function scalarToRawData"
        # cv2.IMREAD_GRAYSCALE is 0
        image = cv2.imread(folder_path+fname, cv2.IMREAD_GRAYSCALE)
        
        left_fit, right_fit, left_lane_inds, right_lane_inds, nonzerox, nonzeroy = find_lines(image)
        
        #image = cv2.imread(folder_path+fname)
        left_curvature, right_curvature, center = radius_curvature(image, left_fit, right_fit)
        print(left_curvature, right_curvature, center)
        #print("= = =" * 3)

#get_radius_curvature(folder_path="../output_images/")

### Sixth, warp the detected lane boundaries back onto the original image

In [13]:
def draw_lanelines_on_image(undist, warped_img, M, left_fit, right_fit, left_curvature, right_curvature, 
                            center, save_filename=''):
    
    ploty = np.linspace(0, warped_img.shape[0]-1, warped_img.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create an image to draw the lane lines on image, if there is not ".astype(np.uint8)"
    # error: /feedstock_root/build_artefacts/opencv_1495915003231/work/opencv-3.2.0/modules/core/src/arithm.cpp:683: error: 
    # (-5) When the input arrays in add/subtract/multiply/divide functions have different types, 
    # the output array type must be explicitly specified in function arithm_op
    warp_zero = np.zeros_like(warped_img).astype(np.uint8)
    #warp_zero = np.zeros_like(warped_img)
    #color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero)) * 255
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    #pts = np.hstack((pts_left, pts_right))
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    
    # Compute the inverse of the M
    Minv = np.linalg.inv(M)
    
    # Warp the blank back to original image space using Minv
    #newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], undist.shape[0]), flags=cv2.INTER_LINEAR)
    
    #scipy.misc.imsave('../output_images/draw_laneline_on_straight_lines1_undist.jpg', undist)
    #scipy.misc.imsave('../output_images/draw_laneline_on_straight_lines1_newwarp.jpg', newwarp)    
    #scipy.misc.imsave('../output_images/draw_laneline_on_straight_lines1_color_warp.jpg', color_warp)
    #print("undist = ", undist)
    #print(undist.shape)
    #print("newwarp = ", newwarp)
    #print(newwarp.shape)
    #print("color_warp = ", color_warp)
    #print(color_warp.shape)
    
    # Combine the result with the original image
    #result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    #print("undist = ", undist)
    #print("newwarp = ", newwarp)
    #undist = undist.astype(np.uint8)  #for gray color
    #print("undist = ", undist)
    #undist = cv2.cvtColor(undist, cv2.COLOR_GRAY2BGR)  #for gray color
    #print("undist = ", undist)
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    #scipy.misc.imsave('../output_images/draw_laneline_on_straight_lines1_only.jpg', result)

    cv2.putText(result, 'Left curvature: {:.0f} m'.format(left_curvature), 
                (50, 50), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)
    cv2.putText(result, 'Right curvature: {:.0f} m'.format(right_curvature), 
                (50, 100), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)
    cv2.putText(result, '{}'.format(center), (50, 150), cv2.FONT_HERSHEY_DUPLEX, 
                1, (255, 255, 255), 2)
    
    return result

def test_draw_lanelines_on_image():
    undist_img =cv2.imread("../output_images/undistort_test4.jpg")

    #if there is not 0, it hits the "error: /feedstock_root/build_artefacts/opencv_1495915003231/work/opencv-3.2.0/modules/core/src/matrix.cpp:1104: error: (-215) cn <= 4 in function scalarToRawData"
    # cv2.IMREAD_GRAYSCALE is 0
    warped_img = cv2.imread("../output_images/birds_eye_view_color_gradient_combined_undistort_test4.jpg", cv2.IMREAD_GRAYSCALE)
    
    #warped_img = cv2.imread("../output_images/birds_eye_view_color_gradient_combined_test4.jpg")
    #print(warped_img.ndim)
    #print("warped_img = ", warped_img)
    
    #warped_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    print(warped_img.ndim)
    print("warped_img = ", warped_img)
    
    left_fit, right_fit, left_lane_inds, right_lane_inds, nonzerox, nonzeroy = find_lines(warped_img, margin=40)
    left_curvature, right_curvature, center = radius_curvature(warped_img, left_fit, right_fit)

    M = perspective_M()
    result = draw_lanelines_on_image(undist_img, warped_img, M, left_fit, right_fit, 
                                 left_curvature, right_curvature, center)

    cv2.imwrite('../output_images/draw_laneline_on_undistort_test4.jpg', result)
    print("Done")

#test_draw_lanelines_on_image()

### Seventh, define the pipeline

In [14]:
# Finds the camera intrinsic and extrinsic parameters from several views of a calibration pattern. 
ret, mtx, dist, rvecs, tvecs = get_camera_parameters()
# Choose a larger odd number to smooth gradient measurements
ksize = 3
# Get the Matrix of perspective
M = perspective_M()

def pipeline(img=""):
    global ret, mtx, dist, rvecs, tvecs
    global M
    global ksize
    
    # Get the undistortion picture
    undist_img = get_undistort(img, mtx, dist, None, mtx)

    # Get the combined image with gradient and color thresh
    binary_img = combined_binary(undist_img, ksize=3, color_thresh=True)
    
    # Get birds eye view image
    w, h = undist_img.shape[1], undist_img.shape[0]
    birds_eye_view_img = get_birds_eye_view(binary_img, M, (w,h), flags=cv2.INTER_LINEAR)
    
    # For debug it
    birds_eye_view_img = birds_eye_view_img.astype(np.uint8)
    
    # Find the lane lines
    left_fit, right_fit, left_lane_inds, right_lane_inds, nonzerox, nonzeroy = find_lines(birds_eye_view_img, 
                                                                                          nwindows=9, margin=30, minpix=500)
    
    # Get radius curvature
    # Maybe need to change the image to gray color image
    left_curvature, right_curvature, center = radius_curvature(birds_eye_view_img, left_fit, right_fit)
    
    # draw the lane lines on the image
    result = draw_lanelines_on_image(undist_img, birds_eye_view_img, M, left_fit, right_fit, 
                                     left_curvature, right_curvature, center)
    #result = draw_lanelines_on_image(binary_img, birds_eye_view_img, M, left_fit, right_fit, 
    #                                 left_curvature, right_curvature, center)

    return result


def test_pipeline(folder_path=""):
    
    test_filename_list = os.listdir(folder_path)
    
    for fname in test_filename_list:
        #m = re.match("undistort_", fname)
        #if m is None:
        #    continue      
        #print("fname = ", fname)
        image = cv2.imread(folder_path+fname)
        result = pipeline(image)
        cv2.imwrite('../output_images/'+'pipeline_'+fname, result)

#test_pipeline(folder_path="../test_images/")
print('Test done')

../camera_cal/calibration5.jpg cannot get the corners.
../camera_cal/calibration4.jpg cannot get the corners.
../camera_cal/calibration1.jpg cannot get the corners.
3 number images cannot get the corners.
Test done


In [15]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
print(1)

1


In [ ]:
last_leftx_base = []
last_rightx_base = []
first_compute_leftx_base = True

last_left_fit = []
last_right_fit = []

#white_output = '../output_images/challenge_10sec.mp4'
white_output = '../output_images/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
#clip1 = VideoFileClip("../challenge_10sec.mp4")
clip1 = VideoFileClip("../project_video.mp4")
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video ../output_images/project_video.mp4
[MoviePy] Writing video ../output_images/project_video.mp4


  9%|▉         | 116/1261 [00:32<05:35,  3.42it/s]